In [1]:
using Distributions
using Optim

In [26]:
function generate_data(N)
    delta1, delta2 = -3 , -2
    beta1, beta2 = [1 4], [2 3]
    rho = -2
    

    Y = zeros(N,2)
    X = zeros(N,2)

    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),2)
        e1,e2 = rand(Logistic(),2)
        lambda = exp(rho*x1)/(1+exp(rho*x1))
        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0

            if false
                println( "$(1*y1) $(1*y2),$(y_init)" )
                println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
                println("-----")
            end

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end

print(generate_data(10))

([0.0 1.0; 1.0 0.0; 1.0 1.0; 0.0 1.0; 1.0 1.0; 0.0 1.0; 0.0 1.0; 1.0 0.0; 0.0 0.0; 1.0 0.0], [-1.18899 1.17363; 1.16994 0.0065057; 0.420236 -0.195954; -0.853918 0.812693; 1.39561 0.786815; -0.359698 -0.691882; -1.11737 -0.292443; -0.188876 -0.317492; -1.05202 -0.832048; 0.986766 -1.37374])

In [22]:
function ll_obj(params;xs=0,ys=0)
    n = size(X)[1]
    k = Integer(size(X)[2]/2) + 1
    
    p = []
    
    for bound=[[0,0],[1,1],[0,1],[1,0]]
        util1 = params[1] .+ xs[:,1].*params[2]  .+ bound[2].*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ bound[1].*params[6]
        prob = exp.(util1).^(bound[1])./(1 .+ exp.(util1))
        prob = prob .* exp.(util2).^(bound[2])./(1 .+ exp.(util2))
        append!(p,[prob])
    end
    
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    lambda = exp.(xs[:,1].*params[7])./(1 .+exp.(xs[:,1].*params[7]))
    ll = p00 .* p[1] + p11 .*p[2]
    ll = ll .+ p01 .* ( lambda .* p[3] .+ (1 .-lambda) .*(1 .- p[1] .- p[2] .- p[4] ) )
    ll = ll .+ p10 .* ( (1 .-lambda) .* p[4] .+ lambda .*(1 .- p[1] .- p[2] .- p[3] ) )

    return -sum(log.(max.(ll,0)))    
end

Y,X = generate_data(1000)
ll_obj([1 4 -3 2 3 -2 -2];xs=X,ys=Y)

576.4129316056142

In [23]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X,ys=Y)
print(result.minimizer)
print(result.minimum)

[1.11192 3.85689 -3.15267 2.11803 3.06966 -2.25419 -4.70891]574.4496439733674

In [8]:
function monte_carlo()
    trials = 400
    params_200 = zeros((7,trials))
    params_1000 = zeros((7,trials))
    for i =1:trials
        Y_200, X_200 = generate_data(200)
        trial_200 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_200,ys=Y_200)
        params_200[:,i] = trial_200.minimizer
        
        Y_1000, X_1000 = generate_data(1000)
        trial_1000 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_1000,ys=Y_1000)
        params_1000[:,i] = trial_1000.minimizer
    end
    return params_200,params_1000
end

results_200,results_1000 = monte_carlo()

([0.865373 1.00586 … 0.672723 2.15223; 4.17021 3.98829 … 4.5247 5.47953; … ; -2.4779 -1.56307 … -2.46137 -1.99946; -0.446059 -4.99168 … 8.89485 -1.62844], [0.897724 0.892906 … 1.0693 0.662592; 4.03238 4.03177 … 3.17235 4.12833; … ; -2.11274 -2.23521 … -1.98561 -2.241; -0.911577 0.167653 … -9.15942 -1.80102])

In [9]:
println(mean(results_200[7,:]))
lambda_200 = exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))
println(lambda_200)
var_200 = std(results_200[7,:]) *exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))^2

-8.829886832405863
0.0001462733953413008


0.009018144775602026

In [10]:
println(mean(results_1000[7,:]))
lambda_1000 = exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))
println(lambda_1000)
var_200 = std(results_1000[7,:]) *exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))^2

-4.748891346514224
0.008586918443880025


0.14389294761101162

In [25]:
function print_results(params_200, params_1000)
       
    #params_200[7,:] = exp.(params_200[7,:])./(1 .+ exp.(params_200[7,:]))
    #params_1000[7,:] = exp.(params_1000[7,:])./(1 .+ exp.(params_1000[7,:]))
    
    println("\\begin{center}")
    println("\\begin{table}")
    println("\\normalsize")
    println("\\begin{tabular}{c|c|c|c|c|c}")
    print("& True & Mean & SD  &  Mean  & SD  \\\\\n")
    print("& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\\\\n")
    println("\\hline \\hline")
    param_names = ["\$ \\alpha_1 \$","\$ \\beta_1 \$", "\$ \\Delta_1 \$", 
        "\$ \\alpha_2 \$", "\$ \\beta_2 \$", "\$ \\Delta_2 \$","\$ \\lambda \$" ]
    true_param = ["1", "4", "-3", "2", "3", "-2", "-2"]
    for i=1:7
        print("$(param_names[i]) &")
        print("$(true_param[i]) &")
        print(" $(round(mean(params_200[i,:]), digits=4)) &")
        print(" $(round(std(params_200[i,:]), digits=3)) &")
        print(" $(round(mean(params_1000[i,:]), digits=4)) &")
        print(" $(round(std(params_1000[i,:]), digits=3)) \\\\\n")
        if (i ==3) | (i==6)
            println( "\\hline \\hline")
        else
            println( "\\hline")
        end
        
    end
    println("\\end{tabular}")
    println("\\end{table}")
    println("\\end{center}")
end

print_results(results_200[:,:], results_1000[:,:])

\begin{center}
\begin{table}
\normalsize
\begin{tabular}{c|c|c|c|c|c}
& True & Mean & SD  &  Mean  & SD  \\
& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\
\hline \hline
$ \alpha_1 $ &1 & 1.0803 & 0.525 & 1.0154 & 0.231 \\
\hline
$ \beta_1 $ &4 & 4.3191 & 0.83 & 4.0614 & 0.358 \\
\hline
$ \Delta_1 $ &-3 & -3.2075 & 0.873 & -3.035 & 0.359 \\
\hline \hline
$ \alpha_2 $ &2 & 2.0882 & 0.484 & 2.0179 & 0.201 \\
\hline
$ \beta_2 $ &3 & 3.1708 & 0.516 & 3.028 & 0.21 \\
\hline
$ \Delta_2 $ &-2 & -2.1247 & 0.637 & -2.0225 & 0.273 \\
\hline \hline
$ \lambda $ &-2 & -8.8299 & 61.662 & -4.7489 & 16.902 \\
\hline
\end{tabular}
\end{table}
\end{center}
